In [48]:
import os, sys, math, datetime
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image
import seaborn as sns

import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow import keras
# from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,DepthwiseConv2D, MaxPooling2D, AvgPool2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
# from tensorflow.keras.layers import ReLU
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
 
# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths, get_file_size
from workbench.utils.utils import parse_model_name, parse_mltk_model_summary
from workbench.data.data import get_lemon_quality_dataset
from workbench.tflite_profiling import get_peak_memory_df, get_tensor_details_df
from workbench.tensorflow import get_layer_details_df

#from dotenv import load_dotenv
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
#import deeplake

%load_ext autoreload



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
%reload_ext autoreload
%autoreload

In [50]:
models_dir = initialize()

In [51]:
automated = False

global model_name

model_name = "mobilenetv2_0.25_96_c3_o2_t5l512.MV1"



In [52]:
base_model_name, alpha, resolution, channels, classes, variation = model_name.split("_")
alpha = float(alpha)
resolution = int(resolution)
classes = int(classes.strip("o"))
channels = int(channels.strip("c"))
PROJECT = "model_DB"

# just needed for INT 8 quantization:
BATCH_SIZE = 32
shuffle_seed = 1

In [53]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [54]:
model = keras.models.load_model(models_tf_path)

In [55]:
# view_model(model, tflite=True, build=True)

# Parse model parameters from MLTK summary

In [56]:
total_params, trainable_params, non_trainable_params, MACs, FLOPs = parse_mltk_model_summary(models_summary_path)        

In [57]:
mltk_model_stats = {}

In [58]:
mltk_model_stats["MACs"] = MACs
mltk_model_stats["FLOPs"] = FLOPs
mltk_model_stats["total_params"] = total_params
mltk_model_stats["trainable_params"] = trainable_params
mltk_model_stats["non_trainable_params"] = non_trainable_params

# Creating tflite Models

In [59]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_saved_model(models_path)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmp1r4sr_tp\assets


In [60]:
# Save the model.
with open(models_tflite_path, "wb") as f:
    f.write(tflite_model)

In [61]:
mltk_model_stats["model_size_kb"] = get_file_size(models_tf_path)
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 1109320
File size in kilobytes is 1083.3203125
File size in bytes is 614960
File size in kilobytes is 600.546875


# tflite with INT8 quantization

In [62]:
dataset_path = Path.cwd().joinpath("datasets", "lemon_dataset")
dataset_path.exists()

True

In [63]:
train_ds, val_ds, test_ds, labels = get_lemon_quality_dataset(dataset_path, resolution, resolution, BATCH_SIZE, channels)

Color mode: rgb
Preparing training dataset...
Found 2021 files belonging to 3 classes.
Preparing validation dataset...
Found 252 files belonging to 3 classes.
Preparing test dataset...
Found 255 files belonging to 3 classes.
Class names: ['bad_quality', 'empty_background', 'good_quality']
Train: (TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Normalize: True


In [64]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]
    
converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)

# set the optimization flag
converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
# enforce integer only quantization
converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_opt.inference_input_type = tf.uint8
converter_opt.inference_output_type = tf.uint8

# provide a representative dataset for quantization
converter_opt.representative_dataset = representative_data_gen

tflite_model_opt = converter_opt.convert()

# Save the model.
with open(models_tflite_opt_path, 'wb') as f:
  f.write(tflite_model_opt)
models_tflite_opt_path

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmp_ey_3jv7\assets


d:\Miniconda\envs\tiny_cnn_6\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


WindowsPath('i:/tinyml/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_INT8.tflite')

In [65]:
mltk_model_stats["tflite_model_INT8_size_kb"] = get_file_size(models_tflite_opt_path)

File size in bytes is 283112
File size in kilobytes is 276.4765625


In [66]:
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 614960
File size in kilobytes is 600.546875


# Profiling tflite models with MLTK profiler

In [67]:
mltk_model_stats["architecture"] = model_name
mltk_model_stats["alpha"] = alpha
mltk_model_stats["img_res"] = resolution
mltk_model_stats["classes"] = classes
mltk_model_stats["channels"] = channels
mltk_model_stats["variation"] = variation

In [68]:
# MLTK profile model reads the mode from a path!

profiling_results = profile_model(str(models_tflite_path), accelerator=None, build=None)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 275968


In [69]:
if automated == False:
    mltk_model_stats["opt_RAM_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [70]:
profiling_results.get_summary(exclude_null=False, full_summary=True)
model_profile = profiling_results.to_dict()

In [71]:
mltk_model_stats["flash_model_size_b_mltk"] = model_profile["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_mltk"] = model_profile["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_mltk"] = model_profile["summary"]["ops"]
mltk_model_stats["macs_mltk"] = model_profile["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_mltk"] = model_profile["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_mltk"] = model_profile["summary"]["energy"]

In [72]:
layers_df = pd.DataFrame.from_dict(model_profile["layers"])
layers_df["name"] = layers_df["index"].map(str) + ": " + layers_df["opcode"]
layers_df.insert(2,"name" , layers_df.pop("name"))
layers_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,conv_2d,0: conv_2d,1050624,497664,4.600492e+06,8.359018e-04,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu6
1,1,depthwise_conv_2d,1: depthwise_conv_2d,387072,165888,2.522040e+06,4.679793e-04,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activatio...
2,2,conv_2d,2: conv_2d,156672,73728,8.374511e+05,1.130917e-04,"1x48x48x8,4x1x1x8,4",1x48x48x4,Padding:same stride:1x1 activation:none
3,3,conv_2d,3: conv_2d,506880,184320,3.653150e+06,6.590034e-04,"1x48x48x4,20x1x1x4,20",1x48x48x20,Padding:valid stride:1x1 activation:relu6
4,4,depthwise_conv_2d,4: depthwise_conv_2d,241920,103680,1.232783e+06,2.410759e-04,"1x48x48x20,1x3x3x20,20",1x24x24x20,Multiplier:1 padding:same stride:2x2 activatio...
...,...,...,...,...,...,...,...,...,...,...
61,61,conv_2d,61: conv_2d,751104,368640,1.252006e+06,8.976863e-05,"1x3x3x80,512x1x1x80,512",1x3x3x512,Padding:same stride:1x1 activation:relu6
62,62,average_pool_2d,62: average_pool_2d,5120,0,4.083712e+04,2.230688e-07,1x3x3x512,1x1x1x512,Padding:valid stride:1x1 filter:3x3 activation...
63,63,reshape,63: reshape,0,0,2.421972e+02,4.810657e-19,"1x1x1x512,2",1x512,Type=none
64,64,fully_connected,64: fully_connected,2048,1024,2.805793e+03,1.749769e-08,"1x512,2x512",1x2,Activation:none


In [73]:
layers_df_path = pathlib.Path.joinpath(models_dir, "layer_df_mltk.pkl")
layers_df.to_pickle(layers_df_path)

In [74]:
mltk_model_stats

{'MACs': '4.071 M',
 'FLOPs': '8.641 M',
 'total_params': 160382,
 'trainable_params': 152654,
 'non_trainable_params': 7728,
 'model_size_kb': 1083.3203125,
 'tflite_model_size_kb': 600.546875,
 'tflite_model_INT8_size_kb': 276.4765625,
 'architecture': 'mobilenetv2_0.25_96_c3_o2_t5l512.MV1',
 'alpha': 0.25,
 'img_res': 96,
 'classes': 2,
 'channels': 3,
 'variation': 't5l512.MV1',
 'opt_RAM_mltk': 275968,
 'flash_model_size_b_mltk': 614960,
 'RAM_runtime_memory_size_b_mltk': 272032,
 'OPS_mltk': 8909690,
 'macs_mltk': 4071184,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.005182527872689136}

In [75]:
# %%capture optimal_runtime_INT
profiling_results_INT8 = profile_model(str(models_tflite_opt_path), accelerator=None, build=False)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 104960


In [76]:
if automated == False:    
    mltk_model_stats["opt_RAM_INT8_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [77]:
# from contextlib import redirect_stdout

# with open("runtime.txt", "w", encoding='utf-8') as f:
#     with redirect_stdout(f):
#         print(profile_model(str(models_tflite_opt_path), accelerator=None, build=False))

In [78]:
profiling_results_INT8.get_summary()

OrderedDict([('name', 'mobilenetv2_0.25_96_c3_o2_t5l512.MV1_INT8'),
             ('accelerator', 'None'),
             ('input_shape', '1x96x96x3'),
             ('input_dtype', 'uint8'),
             ('output_shape', '1x2'),
             ('output_dtype', 'uint8'),
             ('tflite_size', 283112),
             ('runtime_memory_size', 100364),
             ('ops', 9020290),
             ('macs', 4071184),
             ('n_layers', 68),
             ('n_unsupported_layers', 0),
             ('cpu_cycles', 34937859.87586975),
             ('cpu_utilization', 0.0),
             ('cpu_clock_rate', 78000000),
             ('energy', 0.005381788989735626),
             ('j_per_op', 5.966314818853525e-10),
             ('j_per_mac', 1.3219223178651779e-09)])

In [79]:
model_profile_INT8 = profiling_results_INT8.to_dict()


In [80]:
mltk_model_stats["flash_model_size_b__INT8_mltk"] = model_profile_INT8["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_INT8_mltk"] = model_profile_INT8["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_INT8_mltk"] = model_profile_INT8["summary"]["ops"]
mltk_model_stats["macs_INT8_mltk"] = model_profile_INT8["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_INT8_mltk"] = model_profile_INT8["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_INT8_mltk"] = model_profile_INT8["summary"]["energy"]

In [81]:
mltk_model_stats

{'MACs': '4.071 M',
 'FLOPs': '8.641 M',
 'total_params': 160382,
 'trainable_params': 152654,
 'non_trainable_params': 7728,
 'model_size_kb': 1083.3203125,
 'tflite_model_size_kb': 600.546875,
 'tflite_model_INT8_size_kb': 276.4765625,
 'architecture': 'mobilenetv2_0.25_96_c3_o2_t5l512.MV1',
 'alpha': 0.25,
 'img_res': 96,
 'classes': 2,
 'channels': 3,
 'variation': 't5l512.MV1',
 'opt_RAM_mltk': 275968,
 'flash_model_size_b_mltk': 614960,
 'RAM_runtime_memory_size_b_mltk': 272032,
 'OPS_mltk': 8909690,
 'macs_mltk': 4071184,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.005182527872689136,
 'opt_RAM_INT8_mltk': 104960,
 'flash_model_size_b__INT8_mltk': 283112,
 'RAM_runtime_memory_size_b_INT8_mltk': 100364,
 'OPS_INT8_mltk': 9020290,
 'macs_INT8_mltk': 4071184,
 'n_unsupported_layers_INT8_mltk': 0,
 'energy_INT8_mltk': 0.005381788989735626}

In [82]:
layers_INT8_df = pd.DataFrame.from_dict(model_profile_INT8["layers"])
layers_INT8_df["name"] = layers_INT8_df["index"].map(str) + ": " + layers_INT8_df["opcode"]
layers_INT8_df.insert(2,"name" , layers_INT8_df.pop("name"))
layers_INT8_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,quantize,0: quantize,110592,0,9.957829e+05,1.988387e-04,1x96x96x3,1x96x96x3,Type=none
1,1,conv_2d,1: conv_2d,1050624,497664,4.600492e+06,8.359018e-04,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu6
2,2,depthwise_conv_2d,2: depthwise_conv_2d,387072,165888,2.522040e+06,4.679793e-04,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activatio...
3,3,conv_2d,3: conv_2d,156672,73728,8.374511e+05,1.130917e-04,"1x48x48x8,4x1x1x8,4",1x48x48x4,Padding:same stride:1x1 activation:none
4,4,conv_2d,4: conv_2d,506880,184320,3.653150e+06,6.590034e-04,"1x48x48x4,20x1x1x4,20",1x48x48x20,Padding:valid stride:1x1 activation:relu6
...,...,...,...,...,...,...,...,...,...,...
63,63,average_pool_2d,63: average_pool_2d,5120,0,4.083712e+04,2.230688e-07,1x3x3x512,1x1x1x512,Padding:valid stride:1x1 filter:3x3 activation...
64,64,reshape,64: reshape,0,0,2.421972e+02,4.810657e-19,"1x1x1x512,2",1x512,Type=none
65,65,fully_connected,65: fully_connected,2048,1024,2.805793e+03,1.749769e-08,"1x512,2x512",1x2,Activation:none
66,66,softmax,66: softmax,10,0,3.838418e+03,1.652612e-08,1x2,1x2,Type=softmaxoptions


In [83]:
layers_df_INT8_path = pathlib.Path.joinpath(models_dir, "layer_df_INT8_mltk.pkl")
layers_INT8_df.to_pickle(layers_df_INT8_path)

In [84]:
#layers_INT8_df.info()

# Getting peak memory infos

In [85]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")

In [86]:
! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
#! explorer $models_peak_memory_path

Writing model analysis to i:\tinyml\tiny_cnn\models\mobilenetv2_0.25_96_c3_o2_t5l512.MV1\mobilenetv2_0.25_96_c3_o2_t5l512.MV1_peak_memory in CSV format


In [87]:
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")

In [88]:
peak_memory_df = get_peak_memory_df(model_peak_memory_schedule)
peak_memory_df.head(10)

Reading in i:\tinyml\tiny_cnn\models\mobilenetv2_0.25_96_c3_o2_t5l512.MV1\mobilenetv2_0.25_96_c3_o2_t5l512.MV1_peak_memory\execution_schedule_info.csv
Cleaning up the dataframe.


,name,tensor_IDs,RAM_b,operator,index
0,tfl.quantize,"(0, 107)",55296,tfl.quantize,0
1,conv2d_84,"(107, 108)",46080,mobilenetv2/conv1_relu/Relu6;mobilenetv2/conv1...,1
2,block_5_compress_bn,"(108, 109)",36864,mobilenetv2/block_0_dw_relu/Relu6;mobilenetv2/...,2
3,block_0_compress,"(109, 110)",27648,mobilenetv2/block_0_compress_bn/FusedBatchNorm...,3
4,block_1_expand,"(110, 111)",55296,mobilenetv2/block_1_expand_relu/Relu6;mobilene...,4
5,block_1_dw_conv,"(111, 112)",57600,mobilenetv2/block_1_dw_relu/Relu6;mobilenetv2/...,5
6,block_1_compress,"(112, 113)",14976,mobilenetv2/block_1_compress_bn/FusedBatchNorm...,6
7,block_2_expand,"(113, 114)",20736,mobilenetv2/block_2_expand_relu/Relu6;mobilene...,7
8,block_3_dw_bn,"(113, 114, 115)",38016,mobilenetv2/block_2_dw_relu/Relu6;mobilenetv2/...,8
9,block_2_compress,"(113, 115, 116)",24192,mobilenetv2/block_2_compress_bn/FusedBatchNorm...,9


In [89]:
peak_memory = peak_memory_df["RAM_b"].max()
peak_memory

57600

In [90]:
mltk_model_stats["peak_memory_b"] = peak_memory

In [91]:
peak_memory_df["RAM_b"].argmax()

5

In [92]:
tensor_df = get_tensor_details_df(peak_memory_tensor_details)
tensor_df

Reading in i:\tinyml\tiny_cnn\models\mobilenetv2_0.25_96_c3_o2_t5l512.MV1\mobilenetv2_0.25_96_c3_o2_t5l512.MV1_peak_memory\tensor_details.csv
Cleaning up the dataframe.


,id,name,shape,size_b,name_long
0,0,input_1,"(1, 96, 96, 3)",27648,input_1
1,107,tfl.quantize,"(1, 96, 96, 3)",27648,tfl.quantize
2,108,mobilenetv2/conv2d_84/Conv2D,"(1, 48, 48, 8)",18432,mobilenetv2/conv1_relu/Relu6;mobilenetv2/conv1...
3,109,mobilenetv2/block_5_compress_bn/FusedBatchNormV3,"(1, 48, 48, 8)",18432,mobilenetv2/block_0_dw_relu/Relu6;mobilenetv2/...
4,110,mobilenetv2/block_0_compress/Conv2D,"(1, 48, 48, 4)",9216,mobilenetv2/block_0_compress_bn/FusedBatchNorm...
...,...,...,...,...,...
64,170,mobilenetv2/average_pooling2d/AvgPool,"(1, 1, 1, 512)",512,average_pooling2d
65,171,mobilenetv2/flatten/Reshape,"(1, 512)",512,flatten
66,172,mobilenetv2/dense/BiasAdd,"(1, 2)",2,mobilenetv2/dense/MatMul;mobilenetv2/dense/Bia...
67,173,Identity1,"(1, 2)",2,Identity1


# Layer Analytics from TensorFlow Model

In [93]:
layer_details_csv_exists = True
try:
    layer_details_df = get_layer_details_df(models_layer_df_path)
    layer_details_df
except:
    layer_details_csv_exists = False

Reading in i:\tinyml\tiny_cnn\models\mobilenetv2_0.25_96_c3_o2_t5l512.MV1\mobilenetv2_0.25_96_c3_o2_t5l512.MV1_layers.csv


# Logging to Weights & Biases

In [94]:
api = wandb.Api()

# Generate run ids
run_id = wandb.sdk.lib.runid.generate_id()

wandb.init(
    project=PROJECT,
    entity="susbrock",
    name = model_name,
    id=run_id,
    #resume=True
    )

config = wandb.config
config.id = run_id
config.update(mltk_model_stats)


layers_df_wandb = wandb.Table(dataframe=layers_df)
wandb.log({"tflite_layers_mltk" : layers_df_wandb})
layers_INT8_df_wandb = wandb.Table(dataframe=layers_INT8_df)
wandb.log({"tflite_INt8_layers_mltk" : layers_INT8_df_wandb})

peak_memory_df_wandb = wandb.Table(dataframe=peak_memory_df)
wandb.log({"peak_memory" : peak_memory_df_wandb})

tensor_df_wandb = wandb.Table(dataframe=tensor_df)
wandb.log({"tensor_info" : tensor_df_wandb})

if layer_details_csv_exists:
    layer_details_df_wandb = wandb.Table(dataframe=layer_details_df)
    wandb.log({"tensorflow_layer_details" : layer_details_df_wandb})
else:
    pass


wandb.log({"tflite_INt8_layers_plt" : wandb.plot.bar(layers_INT8_df_wandb, "index", "macs", title=model_name)})

wandb.finish()

